In [1]:
import pandas as pd
import os
import json
import ast
from utils import *

In [2]:
pairs = pd.read_csv("trait_condition_pairs.csv")

In [3]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_traits = [normalize_trait(at) for at in all_traits]

In [4]:
rel = pd.read_csv("trait_related_genes.csv")
rel['Related_Genes'] = rel['Related_Genes'].apply(ast.literal_eval)
t2g = pd.Series(rel['Related_Genes'].values, index=rel['Trait']).to_dict()  # the mapping from trait to genes

In [5]:
t2g

{'Atherosclerosis': ['CDKN2A',
  'APOE',
  'NRG1',
  'CDKN2B-AS1',
  'CDKN2B',
  'APOA1',
  'ABO',
  'APOB',
  'LDLR',
  'PCSK9',
  'ABCA1',
  'CETP',
  'TP53',
  'TNF',
  'BCL2',
  'VEGFA',
  'IL6',
  'LPL',
  'TGFB1',
  'MTHFR',
  'SORT1',
  'GCKR',
  'IL10',
  'APOA5',
  'MMP9',
  'CRP',
  'ADIPOQ',
  'APOC3',
  'TLR4',
  'SERPINA1',
  'MGMT',
  'NOS3',
  'PPARG',
  'TNNT2',
  'IGF1',
  'NFKB1',
  'ACE',
  'LIPC',
  'IL1B',
  'CASR',
  'IL17A',
  'SIRT1',
  'PTGS2',
  'PROC',
  'CCL2',
  'PON1',
  'HP',
  'AKT1',
  'TNFAIP3',
  'HIF1A',
  'ESR1',
  'HMOX1',
  'VDR',
  'IL18',
  'ICAM1',
  'PVT1',
  'NLRP3',
  'STAT3',
  'HMGB1',
  'LEP',
  'AGER',
  'GC',
  'SPP1',
  'TLR2',
  'BRCA1',
  'MMP2',
  'VWF',
  'EDN1',
  'BDNF',
  'NPPB',
  'MTOR',
  'G6PC2',
  'CXCR4',
  'SERPINE1',
  'APP',
  'MIR155',
  'TERT',
  'NF2',
  'CXCL8',
  'TNFRSF11B',
  'ADAMTS7',
  'CD14',
  'AR',
  'MIR21',
  'MAPK1',
  'PLTP',
  'ABCB1',
  'CXCL12',
  'FGF23',
  'HLA-DRB1',
  'CTNNB1',
  'COMT',
  'ACE2'

In [6]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output'

suc = 0
condition=None
preformance_ls = []

for trait in all_traits:
    print(f"Trait {trait} only")
    output_dir = os.path.join(output_root, trait)
    os.makedirs(output_dir, exist_ok=True)
    try:
        trait_data, _, _ = select_and_load_cohort(data_root, trait, is_two_step=False)
        redundant_col = ['Age', 'Gender']
        trait_data = trait_data.drop(columns=[redundant_col], errors="ignore")

        Y = trait_data[trait].values
        Z = None
        # Z = trait_data[condition].values
        X = trait_data.drop(columns=[trait]).values

        has_batch_effect = detect_batch_effect(X)
        if has_batch_effect:
            model_constructor = VariableSelection
            fixed_params = {'modified': True}
            tune_params = {'lamda': [3e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3]}
        else:
            model_constructor = Lasso
            fixed_params = {'random_state': 42}
            tune_params = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}

        best_config, best_performance_metrics = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)
        preformance_ls.append(best_performance_metrics)

        model = ResidualizationRegressor(model_constructor, best_config)
        normalized_X, _ = normalize_data(X)
        # normalized_Z, _ = normalize_data(Z)
        model.fit(normalized_X, Y)

        var_names = trait_data.columns.tolist()
        threshold = 0.05
        interpret_result(model, var_names, trait, condition, threshold=threshold, save_output=True,
                         output_dir=output_dir)
        suc += 1
    except:
        print(f"Error processing trait {trait}")
        continue


Trait Breast_Cancer only


/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.889e-03, tolerance: 8.329e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.377e-02, tolerance: 8.652e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.4814814814814815}, 'selection': {'precision': 0.16238663251978122, 'precision_at_50': 0.21600000000000003, 'recall': 0.0295980078263963, 'jaccard': 0.025680724749883633, 'jaccard2': 0.015677627967351327}}
The cross-validation performance: {'prediction': {'accuracy': 0.5078189300411522}, 'selection': {'precision': 0.1686340313547175, 'precision_at_50': 0.16, 'recall': 0.028673070081821416, 'jaccard': 0.02512831351596756, 'jaccard2': 0.015536398037735433}}
The cross-validation performance: {'prediction': {'accuracy': 0.48806584362139915}, 'selection': {'precision': 0.17283218173894704, 'precision_at_50': 0.20799999999999996, 'recall': 0.024759871931696906, 'jaccard': 0.02214322071986304, 'jaccard2': 0.012064627439720477}}
The cross-validation performance: {'prediction': {'accuracy': 0.5358024691358025}, 'selection': {'precision': 0.13087526428671362, 'precision_at_50': 0.17600000000000002, 'recall': 0.010209889718961222, 'ja

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.479e-04, tolerance: 3.000e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.012e-03, tolerance: 2.667e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.3333333333333333}, 'selection': {'precision': 0.0585228232305832, 'precision_at_50': 0.05897565341926243, 'recall': 0.0031026252983293555, 'jaccard': 0.0029544928727906297, 'jaccard2': 0.0}}


...

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.623e-03, tolerance: 8.108e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.338e-03, tolerance: 8.108e-04
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 0.42222222222222217}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.5111111111111111}, 'selection': {'precision': 0.021788254419833366, 'precision_at_50': 0.021788254419833366, 'recall': 0.00808080808080808, 'jaccard': 0.0059594222606560296, 'jaccard2': 0.002666666666666667}}
The cross-validation performance: {'prediction': {'accuracy': 0.4}, 'selection': {'precision': 0.011269841269841268, 'precision_at_50': 0.011269841269841268, 'recall': 0.00404040404040404, 'jaccard': 0.0029962967119290763, 'jaccard2': 0.0028169014084507044}}
The cross-validation performance: {'prediction': {'accuracy': 0.37777777777777777}, 'selection': {'precision': 0.0064516129032258064, 'precision_at_50': 0.0064516129032258064, 'recall': 0.00202020202020202, 'jaccard': 0.0015503875968992248, 'jaccard2': 0.0}}
The cross-validation

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.045e-02, tolerance: 6.795e-03
  model = cd_fast.enet_coordinate_descent(


The gene info file does not contain genes related to the feature 'Type_1_Diabetes_Risk'.
Error processing trait Type_1_Diabetes_Risk
Trait Sjögrens_Syndrome only
Error processing trait Sjögrens_Syndrome
Trait Tourette_Syndrome only
Error processing trait Tourette_Syndrome
Trait Underweight only
Error processing trait Underweight
Trait Psoriatic_Arthritis only
Error processing trait Psoriatic_Arthritis
Trait Social_Behavior only
Error processing trait Social_Behavior
Trait Susceptibility_to_Infections only
Error processing trait Susceptibility_to_Infections
Trait Obstructive_sleep_apnea only
Error processing trait Obstructive_sleep_apnea
Trait Pheochromocytoma_and_Paraganglioma only
Error processing trait Pheochromocytoma_and_Paraganglioma
Trait Uterine_Carcinosarcoma only
The gene info file does not contain genes related to the feature 'Uterine_Carcinosarcoma'.
Error processing trait Uterine_Carcinosarcoma
Trait Stroke only
Error processing trait Stroke
Trait Tuberous_Sclerosis only
Er

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.585e-03, tolerance: 8.108e-04
  model = cd_fast.enet_coordinate_descent(


In [ ]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output'

suc = 0
condition=None
preformance_ls = []

trait = "Breast_Cancer"

print(f"Trait {trait} only")
output_dir = os.path.join(output_root, trait)
os.makedirs(output_dir, exist_ok=True)
trait_data, _, _ = select_and_load_cohort(data_root, trait, is_two_step=False)
redundant_col = ['Age', 'Gender']
trait_data = trait_data.drop(columns=[redundant_col], errors="ignore")

Y = trait_data[trait].values
Z = None
# Z = trait_data[condition].values
X = trait_data.drop(columns=[trait]).values

has_batch_effect = detect_batch_effect(X)
if has_batch_effect:
    model_constructor = VariableSelection
    fixed_params = {'modified': True}
    tune_params = {'lamda': [3e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3]}
else:
    model_constructor = Lasso
    fixed_params = {'random_state': 42}
    tune_params = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}

best_config, best_performance_metrics = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)
preformance_ls.append(best_performance_metrics)

model = ResidualizationRegressor(model_constructor, best_config)
normalized_X, _ = normalize_data(X)
# normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y)

var_names = trait_data.columns.tolist()
threshold = 0.05
interpret_result(model, var_names, trait, condition, threshold=threshold, save_output=True,
                 output_dir=output_dir)
suc += 1



In [ ]:
performance_ls

In [ ]:
[b["selection"]["precision"] for b in performance_ls]

In [ ]:
best_performance_metrics

In [ ]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output'

suc = 0

trait, condition = 'Lung_Cancer', 'Age'
# if trait != 'Heart-rate': continue
output_dir = os.path.join(output_root, trait)
os.makedirs(output_dir, exist_ok=True)


if condition in ['Age', 'Gender']:
    trait_data, _, _ = select_and_load_cohort(data_root, trait, is_two_step=False)
    redundant_col = 'Age' if condition == 'Gender' else 'Gender'
    if redundant_col in trait_data.columns:
        trait_data = trait_data.drop(columns=[redundant_col])

    Y = trait_data[trait].values
    Z = trait_data[condition].values
    X = trait_data.drop(columns=[trait, condition]).values

    has_batch_effect = detect_batch_effect(X)
    if has_batch_effect:
        model_constructor = VariableSelection
        fixed_params = {'modified': True}
        tune_params = {'lamda': [10-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}
    else:
        model_constructor = Lasso
        fixed_params = {'random_state': 42}
        tune_params = {'alpha': [10-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}

    # Tune hyperparameters
    best_config, performance_history = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)

    model = ResidualizationRegressor(model_constructor, best_config)
    normalized_X, _ = normalize_data(X)
    normalized_Z, _ = normalize_data(Z)
    model.fit(normalized_X, Y, normalized_Z)

    var_names = trait_data.columns.tolist()
    threshold = 0.05
    interpret_result(model, var_names, trait, condition, threshold=threshold, save_output=True,
                     output_dir=output_dir)
    suc += 1



In [ ]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output'

suc = 0
for i, (index, row) in enumerate(pairs.iterrows()):
    try:
        trait, condition = row['Trait'], row['Condition']
        # if trait != 'Heart-rate': continue
        output_dir = os.path.join(output_root, trait)
        os.makedirs(output_dir, exist_ok=True)

        if suc >= 20:
            break

        if condition in ['Age', 'Gender']:
            continue
            trait_data, _, _ = select_and_load_cohort(data_root, trait, is_two_step=False)
            redundant_col = 'Age' if condition == 'Gender' else 'Gender'
            if redundant_col in trait_data.columns:
                trait_data = trait_data.drop(columns=[redundant_col])

            Y = trait_data[trait].values
            Z = trait_data[condition].values
            X = trait_data.drop(columns=[trait, condition]).values

            has_batch_effect = detect_batch_effect(X)
            if has_batch_effect:
                model_constructor = VariableSelection
                model_params = {'fixed': {'modified': True}, 'tunable': {'lamda': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}}
            else:
                model_constructor = Lasso
                model_params = {'fixed': {'random_state': 42}, 'tunable': {'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}}

            cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params)

            model = ResidualizationRegressor(model_constructor, model_params)
            normalized_X, _ = normalize_data(X)
            normalized_Z, _ = normalize_data(Z)
            model.fit(normalized_X, Y, normalized_Z)

            var_names = trait_data.columns.tolist()
            threshold = 0.05
            interpret_result(model, var_names, trait, condition, threshold=threshold, save_output=True,
                             output_dir=output_dir)
            suc += 1

        else:
            # if trait != 'Inflammatory_Disorders' or condition != 'Lung_Cancer': continue
            trait_data, condition_data, regressors = select_and_load_cohort(data_root, trait, condition, is_two_step=True, gene_info_path=gene_info_path)
            if regressors is None:
                print(f'No gene regressors for trait {trait} and condition {condition}')
                continue

            print("Common gene regressors for condition and trait", regressors)
            X_condition = condition_data[regressors].values
            Y_condition = condition_data[condition].values

            condition_type = 'binary' if len(np.unique(Y_condition)) == 2 else 'continuous'

            if condition_type == 'binary':
                if X_condition.shape[1] > X_condition.shape[0]:
                    model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
                else:
                    model = LogisticRegression()
            else:
                if X_condition.shape[1] > X_condition.shape[0]:
                    model = Lasso()
                else:
                    model = LinearRegression()

            normalized_X_condition, _ = normalize_data(X_condition)
            model.fit(normalized_X_condition, Y_condition)

            regressors_in_trait = trait_data[regressors].values
            normalized_regressors_in_trait, _ = normalize_data(regressors_in_trait)
            if condition_type == 'binary':
                predicted_condition = model.predict_proba(normalized_regressors_in_trait)[:, 1]
            else:
                predicted_condition = model.predict(normalized_regressors_in_trait)

            trait_data[condition] = predicted_condition
            trait_data = trait_data.drop(columns=regressors)
            trait_data = trait_data.drop(columns=['Age', 'Gender'], errors='ignore')

            Y = trait_data[trait].values
            Z = trait_data[condition].values
            X = trait_data.drop(columns=[trait, condition]).values

            has_batch_effect = detect_batch_effect(X)
            if has_batch_effect:
                model_constructor = VariableSelection
                model_params = {'modified': True, 'lamda': 3e-4}
            else:
                model_constructor = Lasso
                model_params = {'alpha': 1.0, 'random_state': 42}

            cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params)

            model = ResidualizationRegressor(model_constructor, model_params)
            normalized_X, _ = normalize_data(X)
            normalized_Z, _ = normalize_data(Z)
            model.fit(normalized_X, Y, normalized_Z)

            var_names = trait_data.columns.tolist()
            threshold = 0.05
            interpret_result(model, var_names, trait, condition, threshold=threshold, save_output=True,
                             output_dir=output_dir)
            suc += 1
    except Exception as e:
        print(f"Error processing row {i}, for the trait '{trait}' and the condition '{condition}'\n: {e}")
        continue
